### QUORA SPAM FILTER
* Filter spam texts in quora by using Text classification with Deep Learning

In [90]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten
from tensorflow.keras.models import Model, Sequential

In [91]:
file_path = r'D:\\Rishabh\\Python\\DeepLearning\\QuoraSpamFilter\\train.csv'

In [92]:
data = pd.read_csv(file_path, sep=',', names=['qid', 'question_text','target'],header=None)[1:]
data['target'] = data['target'].astype('int32')
data.shape

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1306122, 3)

In [93]:
data.head()

,qid,question_text,target
1,00002165364db923c7e6,How did Quebec nationalists see their province...,0
2,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
3,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
4,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
5,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [94]:
train_data, test_data = train_test_split(data, test_size=0.2,  stratify=data['target'])
train_data.shape , test_data.shape

x_train = train_data['question_text']
y_train  = train_data['target']

x_test = test_data['question_text']
y_test  = test_data['target']

In [95]:
word_tokenize('All is not good ! rishabh')

['All', 'is', 'not', 'good', '!', 'rishabh']

In [96]:
max_len=0
for quest in x_train:
    txt_len = len(word_tokenize(quest))
    if (txt_len > max_len):
        max_len = txt_len

max_len #412

361

In [97]:
sent_len=[]
sent_len = [len(word_tokenize(quest)) for quest in x_train]
print('95% Quantile value {0} and max length is {1}'.format(np.quantile(sent_len, 0.95), np.max(sent_len) ))

95% Quantile value 31.0 and max length is 361


In [98]:
max_len = 35
[val for val in sent_len if val>100]

[147, 155, 361, 111, 123, 169, 108, 182, 125, 101]

In [99]:
tok = Tokenizer(char_level=False, split=' ')
tok.fit_on_texts(x_train)

In [100]:
seq_train = tok.texts_to_sequences(x_train)

In [101]:
vocab_len = len(tok.index_word.keys())
vocab_len

196106

In [102]:
sequence_matrix_train = sequence.pad_sequences(seq_train , maxlen=max_len)

### GloVe Embedding

In [103]:
##Taking all pre trained data in memory
glove_file_path = r'D:\\Rishabh\\Python\\Embeddings\\GloveData\\glove.6B.100d.txt'
embed_dict = {}
with open(glove_file_path, encoding='utf-8') as text:
    for line in text:
        #print(line)
        word, wts  = line.split(maxsplit=1)
        wts        = np.fromstring(wts, 'f',sep=" ")
        embed_dict[word]  = wts

print("Found %s word vectors" % len(embed_dict))

Found 400000 word vectors


In [104]:
## Preparing Embedding matrix
embedding_dim = 100  # As We are using glove data with 100 embeding dimension (100 coefficients of a word) 
num_tokens    = vocab_len+1
embedding_matrix = np.zeros((num_tokens, embedding_dim))
words_not_found = []
word, i, not_in_glove, word_found = None,0, 0, 0

for word , i in tok.word_index.items():
    try:
        ## Replacing word coefficient fro pre trained model with word in train data at specific index
        #embed_vector = embed_dict.get(word) # if using this syntax then need to handle with if embed_vector is None
        embed_vector = embed_dict[word]
        embedding_matrix[i] = embed_vector
        word_found+=1
    except:
        words_not_found.append(word)
        not_in_glove+=1
        
print("Word Found {0} and not found {1}".format(word_found, not_in_glove) )

Word Found 99207 and not found 96899


In [105]:
#words_not_found

In [106]:
def RNN_LSTM(embedding_matrix, embed_dim, max_len, vocab_size):
    inputs = Input(name='inp_layer', shape=[max_len])
    
    layer  = Embedding(vocab_size+1 , embed_dim, input_length=max_len, mask_zero=True,
                      weights=[embedding_matrix],trainable=False)(inputs)
    
    layer  = LSTM(64)(layer)
    layer  = Dense(256, name='FC1', activation='relu')(layer)
    layer  = Dropout(0.4)(layer)
    layer  = Dense(1, name='output_layer', activation='sigmoid')(layer)
    
    model  = Model(inputs=inputs, outputs=layer)
    return model

In [107]:
model = RNN_LSTM(embedding_matrix, embedding_dim, max_len, vocab_len)

In [108]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_layer (InputLayer)       [(None, 35)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 35, 100)           19610700  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 1)                 257       
Total params: 19,669,837
Trainable params: 59,137
Non-trainable params: 19,610,700
__________________________________________

In [109]:
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [110]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [111]:
model.fit(sequence_matrix_train, y_train.values, batch_size=50,
         epochs=15, validation_data=(sequences_matrix_test, y_test.values))

Epoch 1/15
20898/20898 [==============================] - 436s 20ms/step - loss: 0.1339 - accuracy: 0.9491 - val_loss: 0.1149 - val_accuracy: 0.9546
Epoch 2/15
20898/20898 [==============================] - 421s 20ms/step - loss: 0.1099 - accuracy: 0.9566 - val_loss: 0.1092 - val_accuracy: 0.9567
Epoch 3/15
20898/20898 [==============================] - 434s 21ms/step - loss: 0.1037 - accuracy: 0.9589 - val_loss: 0.1075 - val_accuracy: 0.9571
Epoch 4/15
20898/20898 [==============================] - 506s 24ms/step - loss: 0.1005 - accuracy: 0.9601 - val_loss: 0.1088 - val_accuracy: 0.9574
Epoch 5/15
20898/20898 [==============================] - 499s 24ms/step - loss: 0.0964 - accuracy: 0.9612 - val_loss: 0.1123 - val_accuracy: 0.9555
Epoch 6/15
20898/20898 [==============================] - 502s 24ms/step - loss: 0.0945 - accuracy: 0.9617 - val_loss: 0.1096 - val_accuracy: 0.9575
Epoch 7/15
20898/20898 [==============================] - 557s 27ms/step - loss: 0.0922 - accuracy: 0.9627

In [116]:
predictions=model.predict(sequences_matrix_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.9498967786368657